In [ ]:
from torchsig.signals.signal_types import Signal
from torchsig.datasets.dataset_metadata import DatasetMetadata
from torchsig.signals.builders.constellation import ConstellationSignalBuilder
from torchsig.signals.builders.tone import ToneSignalBuilder
import torchsig.transforms.functional as F
from torchsig.utils.dsp import (
    torchsig_complex_data_type,
    frequency_shift,
    multistage_polyphase_resampler
)

import numpy as np
from copy import deepcopy
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
pin = np.linspace(0,10,num=10000)

mean_power_est = 1.0
psat_backoff = 1.0
gain = [0.5, 1.0, 2.0, 4.0, 8.0]

# plot pin/pout
fig = plt.figure(figsize=(8, 8))
for i,g in enumerate(gain):
    psat = mean_power_est * psat_backoff
    scale_factor = psat / g
    pout = psat * np.tanh(pin / scale_factor)
    
    pin_log10 = 10*np.log10(np.where(pin == 0, float('inf'), pin))
    pout_log10 = 10*np.log10(np.where(pout == 0, float('inf'), pout))
    plt.plot(pin_log10, pout_log10)
    
plt.title('Nonlinear Amplifier Model AM/AM across Gains ($P_{sat}$: 0 dBW)',fontsize='large');
plt.xlabel('Pin (dBW)',fontsize='large');
plt.ylabel('Pout (dBW)',fontsize='large');
plt.legend(np.round(10*np.log10(gain),decimals=1), loc='lower right', fontsize='large');
plt.show()

# plot pin/phase_out
phislope = [0.05, 0.1, 0.15, 0.2, -0.05, -0.1, -0.15, -0.2]
phi_origin = 0.5 # fix new y-axis origin at 0.5 (-3 dB)
phimax = 0.1

fig = plt.figure(figsize=(8, 8))
for i,p in enumerate(phislope):
    pin_origin = np.arctanh(phi_origin) # fix new x-axis from template tanh    
    phi = (phimax/2) * (np.tanh( (pin - pin_origin) / p) + np.sign(p))
 
    pin_log10 = 10*np.log10(np.where(pin == 0, float('inf'), pin))
    plt.plot(pin_log10, phi)

plt.title('Nonlinear Amplifier Model AM/PM across Slopes ($\\phi_{0}$, $\\phi_{max}$: (0.05, 0.1))',fontsize='large');
plt.xlabel('Pin (dBW)',fontsize='large');
plt.ylabel('$\\phi$ (rad)',fontsize='large');
plt.legend(np.round(phislope,decimals=2), loc='upper left', fontsize='large');
plt.show()